In [7]:
import tensorflow as tf 
import random
import os ,signal
import numpy as np 
import matplotlib.pyplot as plt
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import img_to_array, load_img
print(tf.__version__)

2.1.0


In [6]:
!wget --no-check-certificate \
    https://storage.googleapis.com/laurencemoroney-blog.appspot.com/horse-or-human.zip \
    -O /tmp/horse-or-human.zip

--2020-05-03 16:07:15--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/horse-or-human.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.167.240
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.167.240|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 149574867 (143M) [application/zip]
Saving to: ‘/tmp/horse-or-human.zip’

/tmp/horse-or-human 100%[===================>] 142.65M  3.96MB/s    in 34s     

2020-05-03 16:07:50 (4.23 MB/s) - ‘/tmp/horse-or-human.zip’ saved [149574867/149574867]



In [8]:
import zipfile

local_zip = '/tmp/horse-or-human.zip'
zip_ref = zipfile.ZipFile(local_zip,'r')
zip_ref.extractall('/tmp/horse-or-human')
zip_ref.close()


In [9]:
train_horse_dir = os.path.join('tmp/horse-or-human/horses')
train_human_dir = os.path.join('tmp/horse-or-human/humans')

train_horse_names = os.listdir(train_horse_dir)
print(train_horse_names[:10])
train_human_names = os.listdir(train_human_dir)
print(train_human_names[:10])


FileNotFoundError: [Errno 2] No such file or directory: 'tmp/horse-or-human/horses'

In [ ]:
nrows = 4
ncols = 4

pic_index = 0

fig = plt.gcf()
fig.set_size_inches(ncols*4,nrows*4)

pic_index += 8

next_horse_pix = [os.path.join(train_horse_dir,fname)
                 for fname in train_horse_names[pic_index - 8:pic_index]]
next_horse_pix = [os.path.join(train_human_dir,fname)
                 for fname in train_human_names[pic_index - 8:pic_index]]


for i , img_path in enumerate (next_horse_pix + next_human_pix):
    sp = plt.subplot(nrows , ncols,i+1)
    sp.axis('Off')
    
    img = mpimg.imread(img_path)
    plt.imshow(img)
    
    plt.show()
    
    

In [ ]:
model = tf.keras.models.Sequential([
    
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(300, 300, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [ ]:
model.summary()

In [ ]:
model.compile(loss = 'binary_crossentropy', optimizer = RMSprop(lr=0.001),
             metrics = ['accuracy'])

DATA PREPROCESSING

In [ ]:
train_datagen = ImageDataGenerator(rescale=1/255)

train_generator = train_datagen.flow_from_directory(
'/tmp/horse-or-human',
target_size = (300,300),
batch_size = 128,
class_mode = 'binary')


In [ ]:
history = model.fit(train_generator, steps_per_epoch = 8,
                   epochs = 15,
                   verbose = 1)

Visualizing Convnet for each layer


In [ ]:
succesive_outputts = [layer.output for layer in model.layers[1:]]

visualization_model = tf.keras.models.Model(inputs = model.input,outputs = successive_outputs)


horse_img_files = [os.path.join(train_horse_dir,f) for f in train_horse_names]
human_img_files = [os.apth.join(train_human_dir,f) for f in train_human_names]
img_path = random.choice(horse_immg_files + human_img_files)


img  = load_img(img_path, traget_size=(300,300))

x = img_to_array(img)
x= x.resape((1,) + x.shape)

x /= 255

successive_feature_maps = visualization_model.predict(x)

layer_names = [laye.name for layer in moddel.layers]

for layer_name , feature_map in zip(layer_names , successive_feature_maps):
    if len(feature_map.shape) == 4:
        n_features = feature_map.shape[-1]
        size = feature_map.shape[-1]
        size = feature_map.shape[1]
        display_grid = np.zeros((size,size,*n_features))
        
        for i in range(n_features):
            
            x = feature_amap[0,:,:,i]
            x -= x.mean()
            x /= x.std()
            x *= 64
            x += 128
            x = np.clip(x,0,255).astype('uint8')
            
            display_grid[:,i*size :(i+1) * size ] = x
        
        
        scale = 20. / n_features
        plt.figure(figsize = (scale *n_features,scale))
        plt.title(layer_name)
        plt.grid(False)
        plt.imshow(display_grid, aspect='auto' , cmap = 'viridis')
        
        
        
